In [1]:
import pandas as pd
import os
from tqdm import tqdm
import seaborn as sns
import plotly.express as px
import math

## Tag extraction

In [11]:
# get file paths

directory = 'data/transcripts'
filepaths_list = []
transcripts_list = []

# iterate over files in that directory
for filename in os.listdir(directory):
    file = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(file) and file.endswith('.csv'):
        filepaths_list.append(file)

# sort transcripts chronologically 
filepaths_list.sort()
print(len(filepaths_list))

117


In [487]:
transcript_df = pd.DataFrame()

for filepath in tqdm(filepaths_list[:]):
    with open(filepath, encoding='utf-8') as file:
        session_df = pd.read_csv(file).drop(columns='Unnamed: 0')
        if session_df.iloc[0]['IdSession'] > 4000:
            transcript_df = pd.concat([transcript_df, session_df])

transcript_df = transcript_df.reset_index(drop=True)
transcript_df

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:05<00:00, 21.70it/s]


,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText
0,3,1,498,"Madame la Présidente de la Confédération, \nSe...",N,19991206,4601,AP-M,1.0,1999-12-06T14:32:24,1999-12-06T14:33:39,NaN,FR
1,1,1,498,La séance d'aujourd'hui s'ouvre sous le signe ...,N,19991206,4601,AP-M,1.0,1999-12-06T14:36:50,1999-12-06T14:50:21,NaN,FR
2,53,6,498,Il appartenait au doyen d'âge de constituer le...,N,19991206,4601,AP-M,1.0,1999-12-06T14:50:21,1999-12-06T14:52:13,NaN,FR
3,52,6,273,Das provisorische Büro hat an seiner Sitzung v...,N,19991206,4601,NaN,1.0,1999-12-06T14:52:13,1999-12-06T15:01:02,b,DE
4,51,6,305,Le Bureau provisoire de notre Assemblée a siég...,N,19991206,4601,NaN,1.0,1999-12-06T15:01:02,1999-12-06T15:07:43,b,FR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
189483,319686,60670,4238,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",N,20230504,5120,BR-F,99.0,2023-05-04T17:04:38,2023-05-04T17:06:20,BR-F,DE
189484,319809,60674,4268,La questione della mediatizzazione dei process...,N,20230504,5120,Mit-F,1.0,2023-05-04T17:06:39,2023-05-04T17:11:27,Mit-F,IT
189485,319699,60674,1122,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",N,20230504,5120,Mit-M,1.0,2023-05-04T17:11:29,2023-05-04T17:11:46,Mit-M,DE
189486,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",N,20230504,5120,Mit-F,1.0,2023-05-04T17:11:46,2023-05-04T17:12:44,Mit-F,IT


In [488]:
with open('data/businesses_tags.csv', encoding='utf-8') as file:
    tags_df = pd.read_csv(file)
    tags_df = tags_df.rename(columns={'Unnamed: 0': 'BusinessNumber'})

tags_list = list(tags_df.columns)
tags_list.remove('BusinessNumber')
print(tags_list)
tags_df

['Éducation', 'Droit', 'Parlement', 'Économie', 'Politique internationale', 'Finances', 'Transports', 'Environnement', 'Santé', 'Politique européenne', 'Questions sociales', 'Politique de sécurité', 'Médias et communication', "Politique d'Etat", 'Science et recherche', 'Agriculture', 'Énergie', 'Aménagement du territoire et logement', 'Politique migratoire', 'Culture', 'Droit civil', 'Emploi et travail', "Droits de l'homme", 'Droit pénal', 'Droit international', 'Fiscalité', 'Protection sociale', 'Justice']


,BusinessNumber,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
0,19970419,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20000086,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20003605,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20003606,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20003647,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18189,20200002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18190,20204399,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18191,20210002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18192,20210401,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [489]:
with open('data/subjects.csv', encoding='utf-8') as file:
    subjects_df = pd.read_csv(file).drop(columns='Unnamed: 0')
    
subjects_df

,IdSubject,BusinessNumber,BusinessShortNumber,Title,TitleFR
0,6688,19850019,85.0190,Utilisation pacifique de l'énergie nucléaire. ...,Utilisation pacifique\nde l'énergie nucléaire....
1,6905,19850019,85.0190,Utilisation pacifique de l'énergie nucléaire. ...,Utilisation pacifique\nde l'énergie nucléaire....
2,645,19850227,85.2270,Droit des assurances sociales,Initiative parlementaire\nMeier Josi.\nDroit d...
3,775,19850227,85.2270,Droit des assurances sociales,Initiative parlementaire\nMeier Josi.\nDroit d...
4,1234,19850227,85.2270,Droit des assurances sociales,Initiative parlementaire\nMeier Josi.\nDroit d...
...,...,...,...,...,...
28708,60010,20237245,23.7245,Pénurie de médicaments. Degré d’efficacité des...,Heure des questions.\nQuestion Feller Olivier....
28709,59975,20237252,23.7252,Le collaborateur de l'OFSP responsable de la g...,Heure des questions.\nQuestion Büchel Roland R...
28710,59999,20237253,23.7253,Chômage en hausse malgré la pénurie de main-d’...,Heure des questions.\nQuestion Strupler Manuel...
28711,59993,20237258,23.7258,Comment les recommandations du comité scientif...,Heure des questions.\nQuestion Python Valentin...


In [492]:
transcript_df = transcript_df[['ID', 'IdSubject', 'Text', 'PersonNumber', 'MeetingCouncilAbbreviation', 'MeetingDate', 'IdSession']]
transcript_subject_df = transcript_df.reset_index().merge(subjects_df[['IdSubject', 'BusinessNumber', 'Title', 'TitleFR']], on='IdSubject', how='left')
transcript_subject_df

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,TitleFR
0,0,3,1,"Madame la Présidente de la Confédération, \nSe...",498,N,19991206,4601,19999004,Ouverture de la législature,Ouverture de la législature
1,1,1,1,La séance d'aujourd'hui s'ouvre sous le signe ...,498,N,19991206,4601,19999004,Ouverture de la législature,Ouverture de la législature
2,2,53,6,Il appartenait au doyen d'âge de constituer le...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,Conseil national.\nVérification des pouvoirs\n...
3,3,52,6,Das provisorische Büro hat an seiner Sitzung v...,273,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,Conseil national.\nVérification des pouvoirs\n...
4,4,51,6,Le Bureau provisoire de notre Assemblée a siég...,305,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,Conseil national.\nVérification des pouvoirs\n...
...,...,...,...,...,...,...,...,...,...,...,...
214764,189483,319686,60670,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",4238,N,20230504,5120,20223026,Préserver les ressources écologiques en régula...,Motion Egger Mike.\nPréserver les ressources é...
214765,189484,319809,60674,La questione della mediatizzazione dei process...,4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,Motion Gysin Greta.\nMédiatisation des procès ...
214766,189485,319699,60674,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",1122,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,Motion Gysin Greta.\nMédiatisation des procès ...
214767,189486,319807,60674,"Collega Fluri, sono molto consapevole del prob...",4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,Motion Gysin Greta.\nMédiatisation des procès ...


In [493]:
transcript_business_df = transcript_subject_df.merge(tags_df, on='BusinessNumber', how='left')
# drop rows where there is no tag
transcript_business_df = transcript_business_df.dropna(subset='Culture')
# set tags columns to int
transcript_business_df[tags_list] = transcript_business_df[tags_list].astype('int')
transcript_business_df

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
2,2,53,6,Il appartenait au doyen d'âge de constituer le...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
3,3,52,6,Das provisorische Büro hat an seiner Sitzung v...,273,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
4,4,51,6,Le Bureau provisoire de notre Assemblée a siég...,305,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
5,5,50,6,"Le Bureau provisoire vous recommande, dans sa ...",498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
6,6,48,6,Chaque conseiller national a communiqué s'il e...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214764,189483,319686,60670,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",4238,N,20230504,5120,20223026,Préserver les ressources écologiques en régula...,...,1,0,0,0,0,0,0,0,0,0
214765,189484,319809,60674,La questione della mediatizzazione dei process...,4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214766,189485,319699,60674,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",1122,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214767,189486,319807,60674,"Collega Fluri, sono molto consapevole del prob...",4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0


In [495]:
# get duplicated transcripts
# this happens when a Transcript is linked to a single Subject, which is linked to more than one Business
# for the majority of cases, a Transcript is linked to a single Subject, which is linked to a single Business

duplicates_df = transcript_business_df.loc[transcript_business_df.duplicated(subset=['ID'], keep=False)]
duplicates_df

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
6568,5505,8477,1642,"Ich werde die Fragen Heim, Bührer und Bezzola,...",134,N,20001204,4605,20005191,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
6569,5505,8477,1642,"Ich werde die Fragen Heim, Bührer und Bezzola,...",134,N,20001204,4605,20005196,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
6570,5505,8477,1642,"Ich werde die Fragen Heim, Bührer und Bezzola,...",134,N,20001204,4605,20005203,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
6571,5506,8476,1642,Die aktuellen Ereignisse seit letzter Woche ha...,399,N,20001204,4605,20005191,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
6572,5506,8476,1642,Die aktuellen Ereignisse seit letzter Woche ha...,399,N,20001204,4605,20005196,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214758,189480,319696,60673,"Die Frage, wie man mit Personen umgeht, die de...",4197,N,20230504,5120,20214536,Réintroduire dans la LAVI la possibilité d'ind...,...,0,0,0,0,0,1,0,0,0,0
214759,189481,319720,60673,Il est clair et juste que les victimes d'infra...,4238,N,20230504,5120,20214533,Réintroduire dans la LAVI la possibilité d'ind...,...,0,0,0,0,0,1,0,0,0,0
214760,189481,319720,60673,Il est clair et juste que les victimes d'infra...,4238,N,20230504,5120,20214534,Réintroduire dans la LAVI la possibilité d'ind...,...,0,0,0,0,0,1,0,0,0,0
214761,189481,319720,60673,Il est clair et juste que les victimes d'infra...,4238,N,20230504,5120,20214535,Réintroduire dans la LAVI la possibilité d'ind...,...,0,0,0,0,0,1,0,0,0,0


In [496]:
# group duplicates by transcript id, then calculate the mean of the topics by group
# if the mean of a topic for a transcript is lower than 1, then it means that not all transcripts in that group share that topic
# for example, 2 transcripts with topic "immigration" but one of them has topic "education" as well, 
# then only topic "immigration" has a mean of 1 and will be kept for these 2 transcripts

mean_df = duplicates_df.groupby('ID')[tags_list].mean()

for col in mean_df.columns:
    mean_df[col] = mean_df[col].apply(lambda x: 0 if x<1 else x)

mean_df = mean_df.astype('int')
mean_df.index.name = None

mean_df

,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,Politique européenne,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
8471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8472,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
319587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
319696,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
319720,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [497]:
mean_duplicates_df = duplicates_df[['ID']].merge(mean_df, right_index=True, left_on='ID', how='left')
mean_duplicates_df = mean_duplicates_df.drop(columns='ID')
mean_duplicates_df

,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,Politique européenne,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
6568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6571,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214758,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
214759,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
214760,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
214761,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [498]:
transcript_processed_df = transcript_business_df.copy()
transcript_processed_df.loc[mean_duplicates_df.index, mean_duplicates_df.columns] = mean_duplicates_df
transcript_processed_df = transcript_processed_df.drop_duplicates(subset='ID')
transcript_processed_df

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
2,2,53,6,Il appartenait au doyen d'âge de constituer le...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
3,3,52,6,Das provisorische Büro hat an seiner Sitzung v...,273,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
4,4,51,6,Le Bureau provisoire de notre Assemblée a siég...,305,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
5,5,50,6,"Le Bureau provisoire vous recommande, dans sa ...",498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
6,6,48,6,Chaque conseiller national a communiqué s'il e...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214764,189483,319686,60670,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",4238,N,20230504,5120,20223026,Préserver les ressources écologiques en régula...,...,1,0,0,0,0,0,0,0,0,0
214765,189484,319809,60674,La questione della mediatizzazione dei process...,4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214766,189485,319699,60674,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",1122,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214767,189486,319807,60674,"Collega Fluri, sono molto consapevole del prob...",4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0


In [499]:
# drop rows without tag (meaning rows where sum of all tag columns = 0)
index_to_drop = transcript_processed_df[tags_list].sum(axis=1).loc[lambda x: x==0].index
print(len(index_to_drop))
transcript_processed_df = transcript_processed_df.drop(index_to_drop)
transcript_processed_df

673


,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
2,2,53,6,Il appartenait au doyen d'âge de constituer le...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
3,3,52,6,Das provisorische Büro hat an seiner Sitzung v...,273,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
4,4,51,6,Le Bureau provisoire de notre Assemblée a siég...,305,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
5,5,50,6,"Le Bureau provisoire vous recommande, dans sa ...",498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
6,6,48,6,Chaque conseiller national a communiqué s'il e...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214764,189483,319686,60670,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",4238,N,20230504,5120,20223026,Préserver les ressources écologiques en régula...,...,1,0,0,0,0,0,0,0,0,0
214765,189484,319809,60674,La questione della mediatizzazione dei process...,4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214766,189485,319699,60674,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",1122,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214767,189486,319807,60674,"Collega Fluri, sono molto consapevole del prob...",4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0


In [517]:
transcript_tags_df = transcript_processed_df.set_index('ID').sort_index()
transcript_tags_df = transcript_tags_df[tags_list]
transcript_tags_df.index.name = 'transcript_id'
transcript_tags_df

,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,Politique européenne,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
transcript_id,,,,,,,,,,,,,,,,,,,,,
48,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319800,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
319801,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
319802,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [518]:
transcript_tags_df.to_csv('data/transcripts_tags.csv', encoding='utf-8')

In [516]:
# number of topics by transcript in percent
round(transcript_processed_df[tags_list].sum(axis=1).sort_values().value_counts(normalize=True)*100,2)

1    61.01
2    24.91
3    10.00
4     3.03
5     0.52
7     0.30
9     0.16
6     0.05
8     0.00
Name: proportion, dtype: float64

## Tag exploration

In [2]:
with open('data/transcripts_tags.csv', encoding='utf-8') as file:
    tags_df = pd.read_csv(file)
    
tags_df

,transcript_id,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
0,48,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,50,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,51,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,52,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,53,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172433,319800,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
172434,319801,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
172435,319802,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
172436,319807,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [3]:
with open('data/persons.csv', encoding='utf-8') as file:
    persons_df = pd.read_csv(file).drop(columns='Unnamed: 0')
    persons_df = persons_df.rename(columns={'GenderAsString': 'gender', 'PartyAbbreviation': 'party'})
    cols_to_keep = ['PersonNumber', 'FirstName', 'LastName', 'gender', 'party']
    persons_df = persons_df[cols_to_keep]
    
    party_names = {
        'PES': 'VERT',
        'VERT-E-S': 'VERT',
        'PRD': 'PLR',
        'PLS': 'PLR',
        'PDC': 'M-E',
        'PBD': 'M-E',
        'CVPO': 'M-E',
        'MCR': 'MCG',
        'pvl': 'PVL',
    }
    persons_df['party'] = persons_df['party'].replace(party_names)
    
persons_df

,PersonNumber,FirstName,LastName,gender,party
0,9,Ruedi,Baumann,m,VERT
1,12,Christine,Beerli,f,PLR
2,14,Duri,Bezzola,m,PLR
3,15,Max,Binder,m,UDC
4,21,Christoph,Blocher,m,UDC
...,...,...,...,...,...
705,4329,Daniel,Ruch,m,PLR
706,4330,Alexandre,Berthoud,m,PLR
707,4331,Marc,Jost,m,PEV
708,4332,Mathilde,Crevoisier Crelier,f,PSS


In [4]:
# get file paths

directory = 'data/transcripts'
filepaths_list = []
transcripts_list = []

# iterate over files in that directory
for filename in os.listdir(directory):
    file = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(file) and file.endswith('.csv'):
        filepaths_list.append(file)

# sort transcripts chronologically 
filepaths_list.sort()
print(len(filepaths_list))

117


In [79]:
transcript_df = pd.DataFrame()

for filepath in tqdm(filepaths_list[:]):
    with open(filepath, encoding='utf-8') as file:
        session_df = pd.read_csv(file).drop(columns='Unnamed: 0')
        if session_df.iloc[0]['IdSession'] > 4700:
            transcript_df = pd.concat([transcript_df, session_df])

transcript_df = transcript_df.reset_index(drop=True)
transcript_df['legislature'] = transcript_df['IdSession'].apply(lambda x: str(x)[:2])
transcript_df['legislature'] = transcript_df['legislature'].astype('int')
transcript_df = transcript_df.rename(columns={'MeetingCouncilAbbreviation': 'council'})
transcript_df['council'] = transcript_df['council'].replace({'N': 'CN', 'S': 'CE'})
transcript_df

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:05<00:00, 20.88it/s]


,ID,IdSubject,PersonNumber,Text,council,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText,legislature
0,38720,6579,311,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,20031201,4701,Mit-M,1.0,2003-12-01T19:00:19,2003-12-01T19:02:57,Mit-M,NaN,47
1,38721,6579,50,"Le mandat de prestations, c'est un contrat. Il...",CN,20031201,4701,BPR-M,99.0,2003-12-01T18:57:39,2003-12-01T19:00:19,BPR-M,FR,47
2,38722,6579,511,Dans un monde dominé par des grandes transform...,CN,20031201,4701,Mit-F,1.0,2003-12-01T18:47:21,2003-12-01T18:57:39,*,FR,47
3,38723,6579,502,"Es ist unbestritten, dass die ETH in den eidge...",CN,20031201,4701,Mit-F,1.0,2003-12-01T18:39:02,2003-12-01T18:47:21,*,DE,47
4,38727,6580,138,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,20031201,4701,Mit-M,1.0,2003-12-01T18:34:36,2003-12-01T18:38:04,Mit-M,NaN,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202681,319800,60663,4268,Quando una coppia si separa non è sempre facil...,CN,20230504,5120,Mit-F,1.0,2023-05-04T16:01:49,2023-05-04T16:06:01,Mit-F,IT,51
202682,319801,60662,4046,"Nel settembre del 2021, ossia quando questa mo...",CN,20230504,5120,Mit-M,1.0,2023-05-04T15:47:52,2023-05-04T15:51:22,Mit-M,IT,51
202683,319802,60640,4268,La convivenza nei centri federali d'asilo è un...,CN,20230504,5120,Mit-F,1.0,2023-05-04T11:28:47,2023-05-04T11:33:13,Mit-F,IT,51
202684,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",CN,20230504,5120,Mit-F,1.0,2023-05-04T17:11:46,2023-05-04T17:12:44,Mit-F,IT,51


In [6]:
with open('data/rapporteurs.csv', encoding='utf-8') as file:
    rapporteurs_df = pd.read_csv(file).drop(columns='Unnamed: 0')

rapporteurs_df

,BusinessNumber,BusinessShortNumber,BusinessTitle,PersonNumber,LastName,FirstName,IdSubject
0,19910411,91.4110,Prestations familiales,1149,Schwaller,Urs,8512
1,19910411,91.4110,Prestations familiales,1149,Schwaller,Urs,8513
2,19910411,91.4110,Prestations familiales,1149,Schwaller,Urs,8563
3,19910411,91.4110,Prestations familiales,1149,Schwaller,Urs,8566
4,19910411,91.4110,Prestations familiales,1149,Schwaller,Urs,9694
...,...,...,...,...,...,...,...
35489,20233445,23.3445,Examen des instruments de la BNS,4213,Bendahan,Samuel,60500
35490,20233446,23.3446,"Reconsidérer et modifier la réglementation ""to...",4008,Landolt,Martin,60500
35491,20233446,23.3446,"Reconsidérer et modifier la réglementation ""to...",4213,Bendahan,Samuel,60500
35492,20233447,23.3447,Analyse des facteurs déterminants susceptibles...,4213,Bendahan,Samuel,60500


In [80]:
weight_groups_dict = {
    'CN': {},
    'CE': {},
}

with open('data/weight_gender_cn.csv', encoding='utf-8') as file:
    weight_groups_dict['CN']['gender'] = pd.read_csv(file).set_index('gender')
    
with open('data/weight_party_cn.csv', encoding='utf-8') as file:
    weight_groups_dict['CN']['party'] = pd.read_csv(file).set_index('party')
    
with open('data/weight_gender_ce.csv', encoding='utf-8') as file:
    weight_groups_dict['CE']['gender'] = pd.read_csv(file).set_index('gender')
    
with open('data/weight_party_ce.csv', encoding='utf-8') as file:
    weight_groups_dict['CE']['party'] = pd.read_csv(file).set_index('party')
    
weight_groups_dict

{'CN': {'gender':             46      47     48      49      50      51
  gender                                               
  f        47.03   51.13   58.4   60.08   65.00   83.45
  m       152.97  148.63  141.6  139.75  134.88  116.49,
  'party':            46     47    48     49     50     51
  party                                          
  AdG      1.00   1.00   0.0   0.00   0.00   0.00
  AdI      1.00   0.00   0.0   0.00   0.00   0.00
  Al       0.00   1.00   1.0   0.00   0.00   1.00
  BastA    0.00   0.00   0.0   0.00   1.00   1.00
  CSPO     0.00   0.00   0.0   0.00   0.62   0.00
  DS       1.00   1.00   0.0   0.00   0.00   0.00
  EàG      0.00   0.00   0.0   0.00   0.00   1.00
  GB       0.00   1.00   1.0   0.00   0.00   0.00
  Lega     2.00   1.00   1.0   2.00   2.00   1.00
  M-E     35.64  29.64  33.7  37.00  33.26  28.00
  MCG      0.00   0.00   0.0   1.00   1.00   0.00
  PCS      1.00   1.00   1.0   0.00   0.00   0.00
  PEV      3.00   3.00   2.0   2.00   2.00   3.00


In [81]:
# drop rows where Text is NaN (only 3 rows)
transcript_df = transcript_df.loc[~transcript_df['Text'].isna()].copy()

# get the length (number of characters) of each transcript
transcript_df['length'] = transcript_df['Text'].apply(lambda x: len(x))

# reduce the length by dividing it by 1000 (max length = +43k, max reduced length = 43)
transcript_df['length_norm'] = transcript_df['length'] / 750

# normalize the reduced length by applying exponential formula (range between 0 and 1)
# length < 500 => weigth < 0.4 // length < 1000 => weigth < 0.6 // length > 3000 => weigth > 0.95
transcript_df['length_weight'] = transcript_df['length_norm'].apply(lambda x: round(1 - math.exp(-x), 2))

transcript_df

,ID,IdSubject,PersonNumber,Text,council,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText,legislature,length,length_norm,length_weight
0,38720,6579,311,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,20031201,4701,Mit-M,1.0,2003-12-01T19:00:19,2003-12-01T19:02:57,Mit-M,NaN,47,664,0.885333,0.59
1,38721,6579,50,"Le mandat de prestations, c'est un contrat. Il...",CN,20031201,4701,BPR-M,99.0,2003-12-01T18:57:39,2003-12-01T19:00:19,BPR-M,FR,47,2211,2.948000,0.95
2,38722,6579,511,Dans un monde dominé par des grandes transform...,CN,20031201,4701,Mit-F,1.0,2003-12-01T18:47:21,2003-12-01T18:57:39,*,FR,47,7594,10.125333,1.00
3,38723,6579,502,"Es ist unbestritten, dass die ETH in den eidge...",CN,20031201,4701,Mit-F,1.0,2003-12-01T18:39:02,2003-12-01T18:47:21,*,DE,47,6100,8.133333,1.00
4,38727,6580,138,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,20031201,4701,Mit-M,1.0,2003-12-01T18:34:36,2003-12-01T18:38:04,Mit-M,NaN,47,511,0.681333,0.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202681,319800,60663,4268,Quando una coppia si separa non è sempre facil...,CN,20230504,5120,Mit-F,1.0,2023-05-04T16:01:49,2023-05-04T16:06:01,Mit-F,IT,51,3697,4.929333,0.99
202682,319801,60662,4046,"Nel settembre del 2021, ossia quando questa mo...",CN,20230504,5120,Mit-M,1.0,2023-05-04T15:47:52,2023-05-04T15:51:22,Mit-M,IT,51,3279,4.372000,0.99
202683,319802,60640,4268,La convivenza nei centri federali d'asilo è un...,CN,20230504,5120,Mit-F,1.0,2023-05-04T11:28:47,2023-05-04T11:33:13,Mit-F,IT,51,4092,5.456000,1.00
202684,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",CN,20230504,5120,Mit-F,1.0,2023-05-04T17:11:46,2023-05-04T17:12:44,Mit-F,IT,51,980,1.306667,0.73


In [82]:
# remove some speaker functions from transcripts (Bundesrat, Bundespräsident, Vize-Präsident Bundesrat, Bundeskanzler, Ratspräsident)
# NB: legislature 46 has 19k transcripts without speaker function, so it was dropped

function_to_drop = [
    'BR-M',
    'BR-F',
    'VPBR-M',
    'VPBR-F',
    'BPR-M',
    'BPR-F',
    'BK-M', 
    'BK-F', 
    'P-M', 
    'P-F',
]
transcript_df = transcript_df.loc[~transcript_df['SpeakerFunction'].isin(function_to_drop)].copy()

# replace typo for a single SpeakerFunction (extra space)
transcript_df['SpeakerFunction'] = transcript_df['SpeakerFunction'].replace({'Mit-F ': 'Mit-F'}).copy()

print('transcripts without function:', transcript_df['SpeakerFunction'].isna().sum())
print('number of transcripts:', len(transcript_df))
transcript_df.query('legislature > 40')['SpeakerFunction'].value_counts()

transcripts without function: 0
number of transcripts: 132164


SpeakerFunction
Mit-M    90947
Mit-F    32903
1VP-M     4627
1VP-F     1565
2VP-M     1407
2VP-F      715
Name: count, dtype: int64

In [83]:
# drop rows where speaker is rapporteur on the subject
transcripts_to_drop = list(transcript_df.merge(rapporteurs_df, on=['IdSubject', 'PersonNumber'], how='inner')['ID'].unique())
print(len(transcripts_to_drop))
transcript_df = transcript_df.loc[~transcript_df['ID'].isin(transcripts_to_drop)].reset_index(drop=True)
transcript_df

28082


,ID,IdSubject,PersonNumber,Text,council,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText,legislature,length,length_norm,length_weight
0,38720,6579,311,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,20031201,4701,Mit-M,1.0,2003-12-01T19:00:19,2003-12-01T19:02:57,Mit-M,NaN,47,664,0.885333,0.59
1,38727,6580,138,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,20031201,4701,Mit-M,1.0,2003-12-01T18:34:36,2003-12-01T18:38:04,Mit-M,NaN,47,511,0.681333,0.49
2,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,20031201,4701,Mit-M,1.0,2003-12-01T15:21:38,2003-12-01T15:23:11,Mit-M,DE,47,366,0.488000,0.39
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,20031201,4701,Mit-M,1.0,2003-12-01T15:04:59,2003-12-01T15:11:33,Mit-M,DE,47,7822,10.429333,1.00
4,38738,6582,311,"Le président (Christen Yves, R, VD): Nous trai...",CN,20031201,4701,Mit-M,1.0,2003-12-01T17:59:30,2003-12-01T17:59:49,Mit-M,FR,47,303,0.404000,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104077,319800,60663,4268,Quando una coppia si separa non è sempre facil...,CN,20230504,5120,Mit-F,1.0,2023-05-04T16:01:49,2023-05-04T16:06:01,Mit-F,IT,51,3697,4.929333,0.99
104078,319801,60662,4046,"Nel settembre del 2021, ossia quando questa mo...",CN,20230504,5120,Mit-M,1.0,2023-05-04T15:47:52,2023-05-04T15:51:22,Mit-M,IT,51,3279,4.372000,0.99
104079,319802,60640,4268,La convivenza nei centri federali d'asilo è un...,CN,20230504,5120,Mit-F,1.0,2023-05-04T11:28:47,2023-05-04T11:33:13,Mit-F,IT,51,4092,5.456000,1.00
104080,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",CN,20230504,5120,Mit-F,1.0,2023-05-04T17:11:46,2023-05-04T17:12:44,Mit-F,IT,51,980,1.306667,0.73


In [84]:
max_length = transcript_df.length.max()
length_list = list(transcript_df['length'])
distrib = dict()
divider = 1000

for i in range(max_length//divider + 1):
    distrib[i] = 0
    
for i in length_list:
    distrib[i//divider] += 1
    
distrib

{0: 37878,
 1: 16374,
 2: 16402,
 3: 15331,
 4: 9674,
 5: 4102,
 6: 1874,
 7: 1036,
 8: 609,
 9: 341,
 10: 186,
 11: 101,
 12: 70,
 13: 42,
 14: 18,
 15: 9,
 16: 10,
 17: 4,
 18: 5,
 19: 4,
 20: 3,
 21: 2,
 22: 2,
 23: 3,
 24: 0,
 25: 0,
 26: 0,
 27: 1,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 1}

In [85]:
transcript_col = ['ID', 'IdSubject', 'PersonNumber', 'Text', 'council', 'IdSession', 
                  'SpeakerFunction', 'LanguageOfText', 'legislature', 'length', 'length_norm', 'length_weight']
transcript_person_df = transcript_df[transcript_col].merge(persons_df, on='PersonNumber', how='left')
transcript_person_df

,ID,IdSubject,PersonNumber,Text,council,IdSession,SpeakerFunction,LanguageOfText,legislature,length,length_norm,length_weight,FirstName,LastName,gender,party
0,38720,6579,311,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,4701,Mit-M,NaN,47,664,0.885333,0.59,Yves,Christen,m,PLR
1,38727,6580,138,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,4701,Mit-M,NaN,47,511,0.681333,0.49,Jean-Philippe,Maitre,m,M-E
2,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,4701,Mit-M,DE,47,366,0.488000,0.39,Christoph,Blocher,m,UDC
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,4701,Mit-M,DE,47,7822,10.429333,1.00,Christoph,Blocher,m,UDC
4,38738,6582,311,"Le président (Christen Yves, R, VD): Nous trai...",CN,4701,Mit-M,FR,47,303,0.404000,0.33,Yves,Christen,m,PLR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104077,319800,60663,4268,Quando una coppia si separa non è sempre facil...,CN,5120,Mit-F,IT,51,3697,4.929333,0.99,Greta,Gysin,f,VERT
104078,319801,60662,4046,"Nel settembre del 2021, ossia quando questa mo...",CN,5120,Mit-M,IT,51,3279,4.372000,0.99,Lorenzo,Quadri,m,Lega
104079,319802,60640,4268,La convivenza nei centri federali d'asilo è un...,CN,5120,Mit-F,IT,51,4092,5.456000,1.00,Greta,Gysin,f,VERT
104080,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",CN,5120,Mit-F,IT,51,980,1.306667,0.73,Greta,Gysin,f,VERT


In [86]:
transcript_tagged_df = transcript_person_df.merge(tags_df, left_on='ID', right_on='transcript_id', how='left')
transcript_tagged_df = transcript_tagged_df.dropna(subset='Culture')
transcript_tagged_df

,ID,IdSubject,PersonNumber,Text,council,IdSession,SpeakerFunction,LanguageOfText,legislature,length,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
0,38720,6579,311,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,4701,Mit-M,NaN,47,664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,38727,6580,138,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,4701,Mit-M,NaN,47,511,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,4701,Mit-M,DE,47,366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,4701,Mit-M,DE,47,7822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38738,6582,311,"Le président (Christen Yves, R, VD): Nous trai...",CN,4701,Mit-M,FR,47,303,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104077,319800,60663,4268,Quando una coppia si separa non è sempre facil...,CN,5120,Mit-F,IT,51,3697,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104078,319801,60662,4046,"Nel settembre del 2021, ossia quando questa mo...",CN,5120,Mit-M,IT,51,3279,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
104079,319802,60640,4268,La convivenza nei centri federali d'asilo è un...,CN,5120,Mit-F,IT,51,4092,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
104080,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",CN,5120,Mit-F,IT,51,980,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [87]:
transcript_filtered_df = transcript_tagged_df.copy()
tags_list = sorted(tags_df.columns)
tags_list.remove('transcript_id')

# join droit droit civil, droit pénal, droit
transcript_filtered_df['Droit'] = transcript_tagged_df['Droit'] + transcript_tagged_df['Droit civil'] + transcript_tagged_df['Droit pénal']
transcript_filtered_df['Droit'] = transcript_filtered_df['Droit'].replace({2: 1, 3: 1})
transcript_filtered_df = transcript_filtered_df.drop(columns=['Droit civil', 'Droit pénal'])

# update tags list
tags_list.remove('Droit civil')
tags_list.remove('Droit pénal')

transcript_filtered_df

,ID,IdSubject,PersonNumber,Text,council,IdSession,SpeakerFunction,LanguageOfText,legislature,length,...,Énergie,Aménagement du territoire et logement,Politique migratoire,Culture,Emploi et travail,Droits de l'homme,Droit international,Fiscalité,Protection sociale,Justice
0,38720,6579,311,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,4701,Mit-M,NaN,47,664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,38727,6580,138,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,4701,Mit-M,NaN,47,511,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,4701,Mit-M,DE,47,366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,4701,Mit-M,DE,47,7822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38738,6582,311,"Le président (Christen Yves, R, VD): Nous trai...",CN,4701,Mit-M,FR,47,303,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104077,319800,60663,4268,Quando una coppia si separa non è sempre facil...,CN,5120,Mit-F,IT,51,3697,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104078,319801,60662,4046,"Nel settembre del 2021, ossia quando questa mo...",CN,5120,Mit-M,IT,51,3279,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
104079,319802,60640,4268,La convivenza nei centri federali d'asilo è un...,CN,5120,Mit-F,IT,51,4092,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
104080,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",CN,5120,Mit-F,IT,51,980,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [88]:
transcript_weighted_df = transcript_filtered_df.copy()
transcript_weighted_df[tags_list] = transcript_weighted_df[tags_list].mul(transcript_weighted_df['length_weight'], axis=0)
transcript_weighted_df

,ID,IdSubject,PersonNumber,Text,council,IdSession,SpeakerFunction,LanguageOfText,legislature,length,...,Énergie,Aménagement du territoire et logement,Politique migratoire,Culture,Emploi et travail,Droits de l'homme,Droit international,Fiscalité,Protection sociale,Justice
0,38720,6579,311,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,4701,Mit-M,NaN,47,664,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
1,38727,6580,138,Eintreten wird ohne Gegenantrag beschlossen\nL...,CN,4701,Mit-M,NaN,47,511,...,0.0,0.00,0.0,0.49,0.0,0.00,0.0,0.0,0.0,0.00
2,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,4701,Mit-M,DE,47,366,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",CN,4701,Mit-M,DE,47,7822,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
4,38738,6582,311,"Le président (Christen Yves, R, VD): Nous trai...",CN,4701,Mit-M,FR,47,303,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104077,319800,60663,4268,Quando una coppia si separa non è sempre facil...,CN,5120,Mit-F,IT,51,3697,...,0.0,0.99,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
104078,319801,60662,4046,"Nel settembre del 2021, ossia quando questa mo...",CN,5120,Mit-M,IT,51,3279,...,0.0,0.00,0.0,0.99,0.0,0.00,0.0,0.0,0.0,0.99
104079,319802,60640,4268,La convivenza nei centri federali d'asilo è un...,CN,5120,Mit-F,IT,51,4092,...,0.0,0.00,1.0,0.00,0.0,1.00,0.0,0.0,0.0,0.00
104080,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",CN,5120,Mit-F,IT,51,980,...,0.0,0.00,0.0,0.00,0.0,0.73,0.0,0.0,0.0,0.00


In [135]:
main_party = ['M-E', 'PLR', 'PSS', 'UDC', 'VERT', 'PVL']
total_members = {'CN': 200, 'CE': 46}
ratio_by = {'tag': (0, 1), 'group': (1, 0)}

In [174]:
council = 'CN'
group = 'party'
legis = 51
axis = ratio_by['tag']

group_weight = weight_groups_dict[council][group][str(legis)]/total_members[council]

# only keep main parties
try:
    group_weight = group_weight[main_party]
except KeyError:
    pass

group_weight

party
M-E     0.1400
PLR     0.1397
PSS     0.1950
UDC     0.2650
VERT    0.1300
PVL     0.0800
Name: 51, dtype: float64

In [175]:
grouped = transcript_weighted_df.query('legislature == @legis & council == @council').groupby(group)[tags_list].sum()
results = grouped.div(grouped.sum(axis=axis[0]), axis=axis[1])

# group weight
results = results.div(group_weight, axis=0)
# normalize
results = results.div(results.sum(axis=0), axis=1)

# transpose
results = results.T

# only keep main parties
try:
    results = results[main_party]
except KeyError:
    pass

results.style.background_gradient(cmap='YlGnBu', axis=None)

party,M-E,PLR,PSS,UDC,VERT,PVL
Agriculture,0.162425,0.093243,0.155306,0.161892,0.241743,0.185392
Aménagement du territoire et logement,0.131955,0.198608,0.224400,0.143413,0.177241,0.124383
Culture,0.138796,0.151469,0.195727,0.182672,0.138260,0.193076
Droit,0.158558,0.137757,0.195660,0.159183,0.157474,0.191368
Droit international,0.140526,0.166267,0.175875,0.227989,0.147821,0.141523
Droits de l'homme,0.101742,0.139704,0.226896,0.188562,0.187296,0.155800
Emploi et travail,0.132439,0.161513,0.259628,0.127701,0.180574,0.138146
Environnement,0.125716,0.119126,0.144069,0.153764,0.260580,0.196746
Finances,0.130194,0.135537,0.198439,0.157333,0.194109,0.184388
Fiscalité,0.152872,0.162387,0.208233,0.124673,0.195275,0.156560


In [61]:
all_transcript_dict = dict()
transcript_temp = transcript_filtered_df.copy()

for i in range(47, 52):
    all_transcript_dict[i] = transcript_temp.loc[transcript_temp['legislature'] == i]

In [62]:
group = {'gender': 'GenderAsString', 'party': 'PartyAbbreviation'}
selected_group = 'gender'
ratio_by_party = True
tag_by_group_dict = dict()

for i, df in all_transcript_dict.items():
    tag_by_group = df.groupby(group[selected_group])[tags_list]

    if ratio_by_party:
        tag_by_group = tag_by_group.loc[:,:].div(tag_by_group.sum(axis=1), axis=0)
    else:
        tag_by_group = tag_by_group.loc[:,:].div(tag_by_group.sum(axis=0), axis=1)
    
    tag_by_group_dict[i] = tag_by_group
    
df.loc[tag_by_group.groups['f']]

AttributeError: 'DataFrameGroupBy' object has no attribute 'loc'

In [58]:
legislature = 51
tag_by_group_dict[legislature].groupby(group[selected_group])[tags_list].mean().T.style.background_gradient(cmap='YlGnBu', axis=0)

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

## Verification

In [461]:
print(transcript_business_df.loc[7521]['ID'])

267141


In [463]:
transcript_id = 267141
transcript_business_df.query('ID == ' + str(transcript_id))

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
7521,6867,267141,49812,Das ist der zweite Teil meiner gestrigen Motio...,4172,S,20200910,5105,20200058,Loi fédérale sur les bases légales des ordonna...,...,1,1,1,0,0,1,0,0,1,0


In [432]:
sample_df = transcript_business_df.loc[transcript_business_df['ID'] == transcript_id][tags_list]
sample_df

,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,Politique européenne,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
1181,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1182,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1183,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1184,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1185,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [433]:
sample_df[sample_df.columns[sample_df.sum() > 0]]

,Finances,Santé,Questions sociales
1181,1,1,0
1182,1,1,0
1183,1,1,0
1184,1,1,0
1185,0,1,1


In [434]:
sample_df[sample_df.columns[sample_df.sum() == len(sample_df)]]

,Santé
1181,1
1182,1
1183,1
1184,1
1185,1
